In [1]:
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys 

In [40]:
def get_all_cats():
    cats=[]
    sub_cat=[]
    urls=[]
    dicto={}
    url = ('https://www.alibaba.com/Products')
    r=requests.get(url)
    web_content = BeautifulSoup(r.text, 'lxml')
    divs = web_content.findAll('div',class_='sub-item')
    for div in divs:
        cat=div.find('h4').find('a').text.replace("\n", "").strip()
        sub_cats=div.findAll('a')
        sub_cats.pop(0)
        for el in sub_cats:
            urls.append(el["href"])
            sub_cat.append(el.text.replace("\n", "").strip())
            cats.append(cat)
    data=[np.array(cats),np.array(sub_cat),np.array(urls)]
    result=pd.DataFrame(data).transpose()
    result.columns = ['Category', 'Sub Category', 'URL']
    return result


In [37]:
#get all pages of the catalog
def get_all_catalog_product(url_catalogue):
    i=1
    urls=[]
    url_catalogue=url_catalogue.replace('pid','cid')
    if (url_catalogue.find('catalog')==-1):
        url_catalogue=url_catalogue[:23]+'catalog/'+url_catalogue[23:]
    r=requests.get(url_catalogue)
    web_content = BeautifulSoup(r.text, 'lxml')
    while (not (web_content.findAll('div',class_='right')) and i<=1):
        url=url_catalogue+'?page='+str(i)
        r=requests.get(url)
        web_content = BeautifulSoup(r.text, 'lxml')
        urls.append(get_all_products(url))
        i=i+1
    return urls
    

In [38]:
#infinite scroll with selenium to get all the products of the catalog
def get_all_products(url):
    urls=[]
    print(url)
    chromedriver_path= "C:/Users/ASUS/Desktop/chromedriver"
    driver = webdriver.Chrome(chromedriver_path)
    driver.get(url)
    driver.find_element_by_tag_name('body').send_keys(Keys.END) 
    time.sleep(10)
    page_source = driver.page_source
    web_content = BeautifulSoup(page_source, 'lxml')
    divs=web_content.findAll('div',class_='offer-list-wrapper')
    for div in divs:
        i=0
        for a in div.findAll('a',href=True):
            if (a["href"][:5]=="//www"):
                if (i%2==0):
                    urls.append(a["href"][2:])
                i=i+1
    return urls

In [149]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
#single product 
names=[]
urls=[]
transactions=[]
quantities=[]
buyers=[]
category=[]
sub_category=[]
for i in range(0,len(list_url)):
    for k in range(0,len(list_url[i][0])):
        url='https://'+list_url[i][0][k]
        r=requests.get(url)
        web_content = BeautifulSoup(r.text, 'lxml')
        spans = web_content.findAll('span',class_='tab-name')
        trouve=False
        for span in spans:
            if (span.text=="Transactions"):
                trouve=True
        if (trouve):
            chromedriver_path= "C:/Users/ASUS/Desktop/chromedriver"
            driver = webdriver.Chrome(chromedriver_path)
            driver.get(url)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.find_element_by_xpath("//span[@title='Transactions']").click()
            #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//span[@title='Transactions']"))).click()
            time.sleep(10)
            page_source = driver.page_source
            web_content = BeautifulSoup(page_source, 'lxml')
            spans = web_content.findAll('span',class_='overview-item-content')
            name= web_content.findAll('h1',class_='module-pdp-title')
            if (len(spans)==3):
                if (spans[2].text>='2'):
                    transactions.append(spans[0].text)
                    quantities.append(spans[1].text)
                    buyers.append(spans[2].text)
                    names.append(name[0].text)
                    urls.append(url)
                    category.append(categories[0])
                    sub_category.append(sub_categories[i])
frames=[urls,names,transactions,quantities,buyers,category,sub_category]
final=pd.DataFrame(frames)
    
        

In [150]:
frames=[urls,names,transactions,quantities,buyers,category,sub_category]
final=pd.DataFrame(frames)
final=final.transpose()
